first of all make the file structure in order to apply keras deeplearning

In [1]:
%mkdir ../data/invasive-species/train/hydrangea
%mkdir ../data/invasive-species/train/nonhydrangea

In [2]:
# make validation folders
%mkdir ../data/invasive-species/valid
%mkdir ../data/invasive-species/valid/hydrangea
%mkdir ../data/invasive-species/valid/nonhydrangea

In [3]:
#make folders for sample
%mkdir ../data/invasive-species/sample
%mkdir ../data/invasive-species/sample/train
%mkdir ../data/invasive-species/sample/train/hydrangea
%mkdir ../data/invasive-species/sample/train/nonhydrangea
%mkdir ../data/invasive-species/sample/valid
%mkdir ../data/invasive-species/sample/valid/hydrangea
%mkdir ../data/invasive-species/sample/valid/nonhydrangea

In [5]:
%mkdir ../data/invasive-species/test/unknown

In [6]:
!tree -d ../data/invasive-species/

../data/invasive-species/
|-- sample
|   |-- train
|   |   |-- hydrangea
|   |   `-- nonhydrangea
|   `-- valid
|       |-- hydrangea
|       `-- nonhydrangea
|-- test
|   `-- unknown
|-- train
|   |-- hydrangea
|   `-- nonhydrangea
`-- valid
    |-- hydrangea
    `-- nonhydrangea

15 directories


In [7]:
import pandas as pd

In [11]:
lab = pd.read_csv('../data/invasive-species/train_labels.csv')

In [12]:
lab.head()

,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1


In [17]:
def makefilename(x):
    return str(x)+'.jpg'

In [18]:
lab['fname'] = lab.name.apply(makefilename)

In [21]:
lab.head()

,name,invasive,fname
0,1,0,1.jpg
1,2,0,2.jpg
2,3,1,3.jpg
3,4,0,4.jpg
4,5,1,5.jpg


In [23]:
invasive = lab.loc[lab['invasive']>0,['fname']]

In [28]:
invasive_file = list(invasive.fname)

In [29]:
noninvasive = lab.loc[lab['invasive']==0,['fname']]

In [30]:
noninvasive_file = list(noninvasive.fname)

In [33]:
from glob import glob
import os

In [32]:
from random import shuffle
import shutil

In [34]:
#move invasive to invasive and noninvasive to noninvasive
from_path = '../data/invasive-species/train/'
to_path = '../data/invasive-species/train/hydrangea/'
for f in invasive_file:
    shutil.move(from_path+f,to_path)

In [35]:
#move invasive to invasive and noninvasive to noninvasive
from_path = '../data/invasive-species/train/'
to_path = '../data/invasive-species/train/nonhydrangea/'
for f in noninvasive_file:
    shutil.move(from_path+f,to_path)

In [36]:
# now make 80:20 split and move it to validation sample
# lets make a function to take care of the repeatative work
def move_file(from_path,to_path,percent):
    all_data = glob(os.path.join(from_path,'*.*'))
    shuffle(all_data)
    split_point = int(len(all_data)*percent)
    #train_data = all_data[:split_point]
    split_data = all_data[split_point:]


    for f in split_data:
        shutil.move(f,to_path)

In [37]:
from_path ='../data/invasive-species/train/hydrangea/'
to_path = '../data/invasive-species/valid/hydrangea/'
move_file(from_path,to_path,.8)

In [38]:
from_path ='../data/invasive-species/train/nonhydrangea/'
to_path = '../data/invasive-species/valid/nonhydrangea/'
move_file(from_path,to_path,.8)

In [39]:
# lets make a function to take care of the repeatative work
def copy_file(from_path,to_path,percent):
    all_data = glob(os.path.join(from_path,'*.*'))
    shuffle(all_data)
    split_point = int(len(all_data)*percent)
    #train_data = all_data[:split_point]
    split_data = all_data[split_point:]
    #now copy these files to the dogs folder in validation set

    for f in split_data:
        shutil.copy(f,to_path)

In [40]:
from_path ='../data/invasive-species/train/nonhydrangea/'
to_path = '../data/invasive-species/sample/train/nonhydrangea/'
move_file(from_path,to_path,.9)

from_path ='../data/invasive-species/train/hydrangea/'
to_path = '../data/invasive-species/sample/train/hydrangea/'
move_file(from_path,to_path,.9)

In [41]:
from_path ='../data/invasive-species/valid/nonhydrangea/'
to_path = '../data/invasive-species/sample/valid/nonhydrangea/'
move_file(from_path,to_path,.9)

from_path ='../data/invasive-species/valid/hydrangea/'
to_path = '../data/invasive-species/sample/valid/hydrangea/'
move_file(from_path,to_path,.9)

In [42]:
ls ../data/invasive-species/train/hydrangea/|wc -l

1042


In [43]:
ls ../data/invasive-species/train/nonhydrangea/|wc -l

609


In [44]:
ls ../data/invasive-species/valid/hydrangea/|wc -l

261


In [45]:
ls ../data/invasive-species/valid/nonhydrangea/|wc -l

153


In [46]:
!tree -d ../data/invasive-species

../data/invasive-species
|-- sample
|   |-- train
|   |   |-- hydrangea
|   |   `-- nonhydrangea
|   `-- valid
|       |-- hydrangea
|       `-- nonhydrangea
|-- test
|   `-- unknown
|-- train
|   |-- hydrangea
|   `-- nonhydrangea
`-- valid
    |-- hydrangea
    `-- nonhydrangea

15 directories


# Use a pretrained model and make prediction

In [52]:
#path = "../data/invasive-species/"
path = "../data/invasive-species/sample/"

## Initially working with sample dataset inorder to check the code and make a quick submission

In [48]:
%matplotlib inline
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [49]:
batch_size = 64

In [50]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [53]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)

Found 184 images belonging to 2 classes.
Found 46 images belonging to 2 classes.


In [54]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
184/184 [==============================] - 12s - loss: 1.0583 - acc: 0.5870 - val_loss: 0.3381 - val_acc: 0.8478


In [55]:
#save the model weights into a results folder
#%mkdir '../data/invasive-species/sample/results'

In [56]:
vgg.model.save_weights(path+'results/ft1.h5')

In [57]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
184/184 [==============================] - 11s - loss: 0.7167 - acc: 0.7228 - val_loss: 0.4645 - val_acc: 0.8043


In [58]:
vgg.model.save_weights(path+'results/ft2.h5')

In [59]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
184/184 [==============================] - 10s - loss: 0.5425 - acc: 0.7826 - val_loss: 0.2699 - val_acc: 0.8478


In [60]:
vgg.model.save_weights(path+'results/ft3.h5')

In [61]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
184/184 [==============================] - 11s - loss: 0.5005 - acc: 0.8152 - val_loss: 0.2772 - val_acc: 0.8913


In [62]:
vgg.model.save_weights(path+'results/ft4.h5')

In [63]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
184/184 [==============================] - 11s - loss: 0.3507 - acc: 0.8587 - val_loss: 0.2972 - val_acc: 0.8696


In [64]:
vgg.model.save_weights(path+'results/ft5.h5')

In [65]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
184/184 [==============================] - 10s - loss: 0.4630 - acc: 0.8424 - val_loss: 0.3769 - val_acc: 0.8696


In [66]:
vgg.fit(batches, val_batches, nb_epoch=2)

Epoch 1/2
184/184 [==============================] - 10s - loss: 0.5233 - acc: 0.8261 - val_loss: 0.3146 - val_acc: 0.8478
Epoch 2/2
184/184 [==============================] - 7s - loss: 0.4662 - acc: 0.8315 - val_loss: 0.2795 - val_acc: 0.8913


# score the data using the above model

In [68]:
batches,pred = vgg.test("../data/invasive-species/" +'test',batch_size = batch_size*2)

Found 1531 images belonging to 1 classes.


In [73]:
filenames = batches.filenames
filenames[:5]

['unknown/779.jpg',
 'unknown/1261.jpg',
 'unknown/878.jpg',
 'unknown/552.jpg',
 'unknown/400.jpg']

In [70]:
pred[:5]

array([[ 0.997 ,  0.003 ],
       [ 0.1028,  0.8972],
       [ 0.3031,  0.6969],
       [ 0.9555,  0.0445],
       [ 0.0414,  0.9586]], dtype=float32)

In [72]:
isinvasive = pred[:,1]

In [74]:
ids = [int(f[8:f.find('.')]) for f in filenames]
ids[:5]

[779, 1261, 878, 552, 400]

In [75]:
subm = np.stack([ids,isinvasive],axis = 1)

In [78]:
np.savetxt('../data/invasive-species/sample/results/subm8_sample.csv',subm,
           fmt = '%d,%.5f',header = 'name,invasive' ,comments = '')

In [77]:
from IPython.display import FileLink
FileLink('../data/invasive-species/sample/results/subm8_sample.csv')

/home/ubuntu/data/invasive-species/sample/results/subm8_sample.csv

# Now let's try the same with training sample

In [79]:
path = "../data/invasive-species/"

In [93]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)

Found 1651 images belonging to 2 classes.
Found 414 images belonging to 2 classes.


In [81]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
1651/1651 [==============================] - 87s - loss: 0.6495 - acc: 0.7547 - val_loss: 0.4157 - val_acc: 0.8527


In [82]:
vgg.fit(batches, val_batches, nb_epoch=4)

Epoch 1/4
1651/1651 [==============================] - 78s - loss: 0.3886 - acc: 0.8534 - val_loss: 0.3496 - val_acc: 0.8768
Epoch 2/4
1651/1651 [==============================] - 62s - loss: 0.3390 - acc: 0.8601 - val_loss: 0.2941 - val_acc: 0.8937
Epoch 3/4
1651/1651 [==============================] - 72s - loss: 0.3365 - acc: 0.8674 - val_loss: 0.3132 - val_acc: 0.8961
Epoch 4/4
1651/1651 [==============================] - 62s - loss: 0.3276 - acc: 0.8740 - val_loss: 0.3200 - val_acc: 0.9010


In [83]:
vgg.model.save_weights(path+'results/ft5.h5')

In [84]:
vgg.model.optimizer.lr = .01

In [85]:
vgg.fit(batches, val_batches, nb_epoch=5)

Epoch 1/5
1651/1651 [==============================] - 79s - loss: 0.3180 - acc: 0.8740 - val_loss: 0.2400 - val_acc: 0.9034
Epoch 2/5
1651/1651 [==============================] - 71s - loss: 0.3094 - acc: 0.8837 - val_loss: 0.2666 - val_acc: 0.8961
Epoch 3/5
1651/1651 [==============================] - 69s - loss: 0.2611 - acc: 0.8928 - val_loss: 0.3087 - val_acc: 0.9058
Epoch 4/5
1651/1651 [==============================] - 71s - loss: 0.2514 - acc: 0.9001 - val_loss: 0.2607 - val_acc: 0.9106
Epoch 5/5
1651/1651 [==============================] - 65s - loss: 0.2484 - acc: 0.8970 - val_loss: 0.2628 - val_acc: 0.9010


In [86]:
batches,pred = vgg.test("../data/invasive-species/" +'test',batch_size = batch_size*2)

Found 1531 images belonging to 1 classes.


In [87]:
filenames = batches.filenames
filenames[:5]
isinvasive = pred[:,1]
ids = [int(f[8:f.find('.')]) for f in filenames]
ids[:5]
subm = np.stack([ids,isinvasive],axis = 1)
np.savetxt('../data/invasive-species/sample/results/invasive_subm_1.csv',subm,
           fmt = '%d,%.5f',header = 'name,invasive' ,comments = '')

In [90]:
vgg.model.load_weights(path+'results/ft5.h5')

In [91]:
vgg.model.optimizer.lr = .01

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=10)

Epoch 1/10
1651/1651 [==============================] - 83s - loss: 0.5686 - acc: 0.7880 - val_loss: 0.4251 - val_acc: 0.8720
Epoch 2/10
1651/1651 [==============================] - 67s - loss: 0.4043 - acc: 0.8595 - val_loss: 0.3260 - val_acc: 0.8865
Epoch 3/10
1651/1651 [==============================] - 66s - loss: 0.3473 - acc: 0.8692 - val_loss: 0.2947 - val_acc: 0.8961
Epoch 4/10
1651/1651 [==============================] - 64s - loss: 0.3457 - acc: 0.8795 - val_loss: 0.2933 - val_acc: 0.8889
Epoch 5/10
1651/1651 [==============================] - 66s - loss: 0.3144 - acc: 0.8849 - val_loss: 0.3039 - val_acc: 0.8889
Epoch 6/10
1651/1651 [==============================] - 63s - loss: 0.2917 - acc: 0.8922 - val_loss: 0.2616 - val_acc: 0.8986
Epoch 7/10
1651/1651 [==============================] - 62s - loss: 0.2901 - acc: 0.8867 - val_loss: 0.3296 - val_acc: 0.8961
Epoch 8/10
1651/1651 [==============================] - 66s - loss: 0.2514 - acc: 0.9049 - val_loss: 0.2894 - val_acc:

In [ ]:
vgg.model.load_weights(path+'results/ft15.h5')

In [ ]:
batches,pred = vgg.test("../data/invasive-species/" +'test',batch_size = batch_size*2)
filenames = batches.filenames
isinvasive = pred[:,1]
ids = [int(f[8:f.find('.')]) for f in filenames]
subm = np.stack([ids,isinvasive],axis = 1)
np.savetxt('../data/invasive-species/results/invasive_subm_15.csv',subm,
           fmt = '%d,%.5f',header = 'name,invasive' ,comments = '')